In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re
import gc
import time
import warnings
warnings.filterwarnings("ignore")
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from collections import defaultdict 
from tqdm import tqdm
from common_val import *

### 公共函数

In [3]:
def feature_list_split(x):
    feature_dict = defaultdict(list)
    for fea in x.split('\x01'):
        field = re.split('\x02', fea)
        #fea = re.split('\x03', field[1])
        #feature_dict[field[0]].append({'feature_id':fea[0], 'value':fea[1]})
        feature_dict[field[0]].append(field[1])
    return feature_dict

In [7]:
user_features = ['101','121','122','124','125','126','127', '128', '129', '150_14', '127_14', '109_14', '110_14']
item_features = ['205','206','207', '210','216','508','509', '702', '853', '301']

## 公共特征预处理

In [4]:
fea_cols = ['common_feature_index', 'feature_num2', 'feature_list2']
fea_train = pd.read_csv(common_features_train_csv, header=None, names=fea_cols, nrows=10000)
fea_train.shape

(10000, 3)

In [5]:
fea_train['fea_dict2'] =  fea_train['feature_list2'].map(feature_list_split)

In [ ]:
fea_train['101'] = fea_train['fea_dict2'].map(lambda x: x['101']) # 用户ID

fea_train['121'] = fea_train['fea_dict2'].map(lambda x: x['121']) # 用户的一种分类ID
fea_train['122'] = fea_train['fea_dict2'].map(lambda x: x['122']) # 用户的一种分类ID
fea_train['124'] = fea_train['fea_dict2'].map(lambda x: x['124']) # 用户性别分类ID

fea_train['125'] = fea_train['fea_dict2'].map(lambda x: x['125']) # 用户年龄分类ID
fea_train['126'] = fea_train['fea_dict2'].map(lambda x: x['126']) # 用户消费水平分类I
fea_train['127'] = fea_train['fea_dict2'].map(lambda x: x['127']) # 用户消费水平分类II
fea_train['128'] = fea_train['fea_dict2'].map(lambda x: x['128']) # 用户是否就业
fea_train['129'] = fea_train['fea_dict2'].map(lambda x: x['129']) # 用户地理信息分类ID

fea_train['150_14'] = fea_train['fea_dict2'].map(lambda x: x['150_14']) # 用户意图ID以及用户在该意图上的历史行为累积数量* ：多值
fea_train['127_14'] = fea_train['fea_dict2'].map(lambda x: x['127_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*
fea_train['109_14'] = fea_train['fea_dict2'].map(lambda x: x['109_14']) # 商品类目ID以及用户在该类目上的历史行为累积数量*
fea_train['110_14'] = fea_train['fea_dict2'].map(lambda x: x['110_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*

In [12]:
fea_train['userid'] = fea_train['fea_dict2'].map(lambda x: x['101'][0].split('\x03')[0] if '101' in x else -1) # 用户ID


In [15]:
fea_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   common_feature_index  10000 non-null  object
 1   feature_num2          10000 non-null  int64 
 2   feature_list2         10000 non-null  object
 3   fea_dict2             10000 non-null  object
 4   userid                10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [14]:
fea_train['userid'].max()

TypeError: '>=' not supported between instances of 'str' and 'int'